# Obtención de la Información.
Una vez descargados los archivos, y colocados en su carpeta correspondiente podemos comenzar con la formateo, es decir, obtener las distintas tablas que vienen dentro de cada hoja.


Ahora, procedemos a llamar el documento recien creado. 

In [76]:
import pandas as pd

Sonora = pd.read_excel('..\\medium_data\\book_son.xlsx',sheet_name=None)
Municipios = list(Sonora.keys())


Ahora, debemos separar por bloques. Primero eliminamos la primer y segunda columna, son irrelevantes, tambien las columnas en blanco entre medias.  

In [77]:
data = Sonora['Aconchi']
data.head()

df = data.copy()
df.drop(columns=["Unnamed: 0.1","Unnamed: 0"], inplace=True)
df.dropna(how = 'all', axis = 0, inplace=True)
df.reset_index(drop=True,inplace=True)

abs_index = []
for i in range(len(df)):
    if df.iloc[i, 1] == 'Absoluto':
        abs_index.append(i)
abs_index.append(len(df))

Finalmente revisamos. 

In [78]:
new_dfs = []
for a in range(len(abs_index) - 1):
    u = df.iloc[abs_index[a]:abs_index[a + 1],:]
    u = u.dropna(how = 'all', axis = 1) 
    new_dfs.append(u)

Terminaremos aplicando todo a los 72 municipios. 

In [79]:
df = data.copy()

def transform(mun: str):
    df = Sonora[mun].copy()
    df.drop(columns=["Unnamed: 0.1","Unnamed: 0"], inplace=True)
    df.dropna(how = 'all', axis = 0, inplace=True)
    df.reset_index(drop=True,inplace=True)

    abs_index = []
    for i in range(len(df)):
        if df.iloc[i, 1] == 'Absoluto':
            abs_index.append(i)
    abs_index.append(len(df))
    new_dfs = []
    for a in range(len(abs_index) - 1):
        u = df.iloc[abs_index[a]:abs_index[a + 1],:]
        u = u.dropna(how = 'all', axis = 1) 
        new_dfs.append(u)
    return new_dfs

transform_muns = {mun : transform(mun) for mun in Municipios}